In [2]:
import numpy as np
import pandas as pd
# Just to plot the time series and see what it looks like
import matplotlib.pyplot as plt

%load_ext rpy2.ipython

import rpy2.robjects as ro
from rpy2.robjects.packages import importr
import os

def load_smooth_dev():
    """Load the smooth package in development mode"""
    # Path to your smooth package
    smooth_path = "~/Cursor/smooth/"  # Adjust this path if needed
    
    # Load devtools and then load_all() the smooth package
    ro.r(f'''
    if (!requireNamespace("devtools", quietly=TRUE)) {{
        install.packages("devtools", repos="https://cran.rstudio.com/")
    }}
    devtools::load_all("{smooth_path}")
    ''')
    
    print("Smooth package loaded in development mode")

# Load the package in development mode
load_smooth_dev()

ℹ Loading smooth


R[write to console]: Loading required package: greybox

R[write to console]: Package "greybox", v2.0.4.41004 loaded.


R[write to console]: This is package "smooth", v4.1.2.41005


R[write to console]: In addition: 
R[write to console]: Warning message:

R[write to console]: In (function (package, help, pos = 2, lib.loc = NULL, character.only = FALSE,  :
R[write to console]: 
 
R[write to console]:  libraries ‘/usr/local/lib/R/site-library’, ‘/usr/lib/R/site-library’ contain no packages



Smooth package loaded in development mode


In [3]:
#### Global level and ETS(A,N,N) ####
np.random.seed(33)  # For reproducibility
n_points = 100
time_series = np.random.normal(100, 10, n_points)  # Random errors
#dates = pd.date_range(start='2023-01-01', periods=n_points, freq='ME')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series})

In [4]:
%%R -i ts_df

model <- adam(ts_df, model = "ANN", lags = c(12))

forecast(model, h = 12)

Time Series:
Start = 101 
End = 112 
Frequency = 1 
 [1] 99.61957 99.61957 99.61957 99.61957 99.61957 99.61957 99.61957 99.61957
 [9] 99.61957 99.61957 99.61957 99.61957


In [5]:
#### Local level and ETS(A,N,N) ####
np.random.seed(41)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + errors[i+1]
#dates = pd.date_range(start='2023-01-01', periods=n_points, freq='ME')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series})

In [6]:
%%R -i ts_df

# ETS(A,N,N) with alpha=0.1
model <- adam(ts_df, model = "ANN", lags = c(12), persistence = 0.1)

cat("ETS(A,N,N) with alpha=0.1:\n")
forecast(model, h = 12) |> print()

# ETS(A,N,N) with estimated alpha
model <- adam(ts_df, model = "ANN", lags = c(12))
cat("\nETS(A,N,N) with estimated alpha:\n")
print(model$persistence)
forecast(model, h = 12) |> print()


ETS(A,N,N) with alpha=0.1:
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 83.22599 83.22599 83.22599 83.22599 83.22599 83.22599 83.22599 83.22599
 [9] 83.22599 83.22599 83.22599 83.22599

ETS(A,N,N) with estimated alpha:
     alpha 
0.06531787 
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 84.70888 84.70888 84.70888 84.70888 84.70888 84.70888 84.70888 84.70888
 [9] 84.70888 84.70888 84.70888 84.70888


In [7]:
#### Local trend and ETS(A,A,N) ####
np.random.seed(42)  # For reproducibility
n_points = 120
errors = np.random.normal(0, 10, n_points)  # Random errors
trend = np.random.normal(0.5, 2, n_points)  # Random trend
time_series = np.zeros(n_points)
time_series[0] = 100
for i in range(n_points-1):
    # This is SES with alpha=0.1, just in an alternative form
    time_series[i+1] = time_series[i] + (0.1-1) * errors[i] + trend[i] + errors[i+1]
#dates = pd.date_range(start='2023-01-01', periods=n_points, freq='ME')  # Monthly frequency
ts_df = pd.DataFrame({'value': time_series})


In [13]:
%%R -i ts_df

# ETS(A,A,N)
model1 <- adam(ts_df, model = "AAN", lags = c(12), persistence = c(0.1, 0.02))

cat("ETS(A,A,N):\n")
forecast(model1, h = 12) |> print()

# ETS(A,Ad,N) estimated
cat("\nETS(A,Ad,N) estimated parameters:\n")
print(adam(ts_df, model = "AAdN", lags = c(12))$B)

# ETS(A,Ad,N)
model2 <- adam(ts_df, model = "AAdN", lags = c(12), persistence = c(0.1, 0.02))

cat("\nETS(A,Ad,N):\n")
forecast(model2, h = 12) |> print()

ETS(A,A,N):
Time Series:
Start = 121 
End = 132 


Frequency = 1 
 [1] 165.2555 165.7640 166.2724 166.7808 167.2892 167.7977 168.3061 168.8145
 [9] 169.3230 169.8314 170.3398 170.8483

ETS(A,Ad,N) estimated parameters:
       alpha         beta          phi        level        trend 
  0.16827523   0.01042486   0.98630131 104.77485790  -0.38092112 

ETS(A,Ad,N):
Time Series:
Start = 121 
End = 132 
Frequency = 1 
 [1] 163.2357 163.7013 164.1408 164.5555 164.9469 165.3163 165.6649 165.9939
 [9] 166.3044 166.5974 166.8740 167.1350
